In [3]:
from HAMILTONIAN import Hamiltonian
import os
import re
import json
import argparse
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import calculations as calc
import PAI as pai
from HAMILTONIAN import Hamiltonian
from scipy.stats import binom
from plotting import calcOverhead
import math
from fractions import Fraction
from typing import Tuple, Optional
import time
from concurrent.futures import ProcessPoolExecutor, as_completed
from main import TE_PAI


In [14]:
q = 20
Δ = np.pi / (2**11)
T = 1
N = 1000
n_snap = 1
rng = np.random.default_rng(0)
freqs = rng.uniform(-1, 1, size=q)
hamil = Hamiltonian.spin_chain_hamil(q, freqs, j=1.0)
tepai = TE_PAI(hamil,q,Δ,T,N,n_snap)
tepai_count = tepai.expected_num_gates
print("TEPAI number of gates:", tepai_count)

The number of qubit:20
Number of terms in the Hamiltonian:80
TEPAI number of gates: 93153.0285181921


In [15]:
# New number of Trotter steps to match TEPAI gate count
N = 1164

times = np.linspace(0, T, N)
terms = [hamil.get_term(t) for t in times]
gates = []
n = int(N / n_snap)

for i in range(N):
    if i % n == 0:
        gates.append([])
    gates[-1] += [
        (pauli, 2 * coef * T / N, ind)
        for (pauli, ind, coef) in terms[i]
    ]
print("Trotter gates:", sum(len(g) for g in gates))

Trotter gates: 93120
